<font size="5">
 <div class="alert alert-block alert-info"><b>Master in Data Science - Iscte <b>
     </div>
</font> 
 
 
     
    
  <font size="5"> OEOD </font>
  
  
  
  <font size="3"> **Diana Aldea Mendes**, October 2023 </font>
  
   
  <font size="3"> *diana.mendes@iscte-iul.pt* </font> 
  
    
 
  
    
  <font color='blue'><font size="5"> <b>Week 6 - Gym-AnyTrading<b></font></font>


#########################################################

- More info here: https://github.com/AminHP/gym-anytrading/tree/master 
- *OpenAI Gym* environments for **reinforcement learning-based trading algorithms**.
- Trading algorithms are mostly implemented in two markets: FOREX and Stock.
- Three Gym environments: TradingEnv, ForexEnv, and StocksEnv.
    - TradingEnv is an abstract environment which is defined to support all kinds of trading environments. 
    - ForexEnv and StocksEnv are simply two environments that inherit and extend TradingEnv. 

# Instalation

In [ ]:
#pip install gym-anytrading

In [ ]:
#pip install stable_baselines3

## Example 1

In [ ]:
import gymnasium as gym
import gym_anytrading
import matplotlib.pyplot as plt

env = gym.make('forex-v0')
# env = gym.make('stocks-v0')

In [ ]:
# create environment with specific parameters
# mu = pd.read_csv("MU.csv")
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL

custom_env = gym.make('forex-v0',
    # df = mu
    df=FOREX_EURUSD_1H_ASK,
    window_size=10,
    frame_bound=(50, 100),
    unit_side='right'
)

In [ ]:
# print information
print("env information:")
print("> shape:", env.unwrapped.shape)
print("> df.shape:", env.unwrapped.df.shape)
print("> prices.shape:", env.unwrapped.prices.shape)
print("> signal_features.shape:", env.unwrapped.signal_features.shape)
print("> max_possible_profit:", env.unwrapped.max_possible_profit())

print()
print("custom_env information:")
print("> shape:", custom_env.unwrapped.shape)
print("> df.shape:", custom_env.unwrapped.df.shape)
print("> prices.shape:", custom_env.unwrapped.prices.shape)
print("> signal_features.shape:", custom_env.unwrapped.signal_features.shape)
print("> max_possible_profit:", custom_env.unwrapped.max_possible_profit())

In [ ]:
## plot environment
env.reset()
env.render()

# Short and Long positions are shown in red and green colors.

In [ ]:
### complete example with update rule

from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL


env = gym.make('forex-v0', frame_bound=(50, 100), window_size=10)
# env = gym.make('stocks-v0', frame_bound=(50, 100), window_size=10)

observation = env.reset(seed=2023)
while True:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    # env.render()
    if done:
        print("info:", info)
        break

plt.cla()
env.unwrapped.render_all()
plt.show()

In [ ]:
# You can use render_all method to avoid rendering on each step and prevent time-wasting.
# As you see, the first 10 points (window_size=10) on the plot don't have a position. 
# Because they aren't involved in calculating reward, profit, etc. 
# They just display the first observations. So the environment's _start_tick and initial _last_trade_tick are 10 and 9.

## Example 2

### Imports

In [ ]:
#!pip install quantstats

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import Actions

from stable_baselines3 import A2C

import quantstats as qs

### Create Env

In [ ]:
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()

window_size = 10
start_index = window_size
end_index = len(df)

env = gym.make(
    'stocks-v0',
    df=df,
    window_size=window_size,
    frame_bound=(start_index, end_index)
)

print("observation_space:", env.observation_space)

### Train Env

In [ ]:
env.reset(seed=2023)
model = A2C('MlpPolicy', env, verbose=0)
model.learn(total_timesteps=1_000)

### Test Env

In [ ]:
action_stats = {Actions.Sell: 0, Actions.Buy: 0}

observation, info = env.reset(seed=2023)

while True:
    # action = env.action_space.sample()
    action, _states = model.predict(observation)

    action_stats[Actions(action)] += 1
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    # env.render()
    if done:
        break

env.close()

print("action_stats:", action_stats)
print("info:", info)

### Plot Results

In [ ]:
plt.figure(figsize=(16, 6))
env.unwrapped.render_all()
plt.show()

### Analysis Using `quantstats`

In [ ]:
qs.extend_pandas()

net_worth = pd.Series(env.unwrapped.history['total_profit'], index=df.index[start_index+1:end_index])
returns = net_worth.pct_change().iloc[1:]

qs.reports.full(returns)
qs.reports.html(returns, output='SB3_a2c_quantstats.html')